# **Stable Diffusion v1.4 — Text-to-Image Generation**

**Huggingface**의 `diffuser` 라이브러리를 사용하여 **Stable Diffusion v1.4** 모델로 다양한 이미지를 직접 생성해 봅니다.

## 학습 목표
- Stable Diffusion 추론 파이프라인의 구조적 이해: 사전 학습된 모델을 로드하고, 텍스트 프롬프트가 이미지 텐서로 변환되는 전체 과정을 파악한다.
- 생성 파라미터 최적화를 통한 결과 제어
    - 핵심 하이퍼파라미터 변화가 이미지의 품질과 프롬프트 준수율에 미치는 영향을 실험하고 최적의 조합을 도출한다.

---

### 구성 개요

1. Stable Diffusion 모델 로드
2. 텍스트 프롬프트로 이미지 생성
3. 창의성
4. Seed
5. Negative Prompt
6. Inference step
7. Style variation
8. 해상도와 구도

---
## 0) Setup (환경 준비)
- **목적:** 실습에 필요한 패키지 설치

In [ ]:
import importlib.util
import subprocess
import sys

def ensure_package(pkg_name: str, import_name=None):
    """
    패키지 설치 여부를 확인하고, 설치되어 있지 않으면 설치합니다.
    """
    name = import_name or pkg_name
    # 패키지가 설치되어 있는지 확인
    if importlib.util.find_spec(name) is None:
        print(f"[install] {pkg_name} 라이브러리를 설치 중입니다... (import name: {name})")
        try:
            # -q 옵션을 추가하여 설치 과정을 간결하게 유지할 수 있습니다.
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg_name])
            print(f"[success] {pkg_name} 설치 완료.")
        except subprocess.CalledProcessError as e:
            print(f"[error] {pkg_name} 설치 실패: {e}")
    else:
        print(f"[ok] {pkg_name} 이미 설치되어 있습니다.")

# 설치가 필요한 패키지 리스트 (패키지명, 임포트명)
# 임포트명이 패키지명과 다른 경우 튜플로 지정합니다.
packages = [
    ("diffusers", "diffusers"),
    ("transformers", "transformers"),
    ("accelerate", "accelerate")
]

# 루프를 돌며 확인 및 설치
for pkg, imp in packages:
    ensure_package(pkg, imp)


---
## 0-1) 환경 세팅
- **목적:** 실습에 필요한 Hugging Face Access Token을 발급받고 로그인 합니다.
- **관찰 포인트**
    - Hugging Face Access Token 발급
       - Hugging Face 우상단 프로필 아이콘 → Settings → Access Tokens.
       - New token 클릭
       - 이름: 원하는 이름 입력
       - Role: Read (기본값이면 OK)
       - 생성하면 hf_~~~로 시작하는 긴 문자열이 나오는데, 이걸 어딘가에 복사해 두세요.
    - Huggingface 계정 로그인 (access token 필요)

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()
# access_token = "hf_여기에_본인의_토큰_입력"

---
## 1) Stable Diffusion 모델 로드
- **목적:** Stable Diffusion 1.4 모델을 로드합니다
- **관찰 포인트**
    - model_id는 우리가 사용할 모델의 ID이며
    - pipe는 StableDiffusionPipeline을 통해 모델을 GPU에 로드

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
from IPython.display import display

# 모델 로드 (Stable Diffusion v1-4)
model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    # use_auth_token=access_token, # 토큰 직접 전달
    torch_dtype=torch.float16   # 메모리 절약을 위해 권장
).to("cuda")

# 파이프라인 내부 구성 요소 출력
for name, component in pipe.components.items():
    print(f"{name}: {type(component).__name__}")

---
## 1-1) 파이프라인 구성 요소와 파라미터 매핑

<table style="margin-left: 0; margin-right: auto;">
    <thead>
        <tr>
            <th style="text-align: left;">파라미터명</th>
            <th style="text-align: left;">연관 내부 객체</th>
            <th style="text-align: left;">역할 설명</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><b>prompt</b></td>
            <td>tokenizer, text_encoder</td>
            <td>입력한 문장을 수치화하여 이미지의 <b>주제와 방향</b>을 결정</td>
        </tr>
        <tr>
            <td><b>negative_prompt</b></td>
            <td>text_encoder</td>
            <td>이미지에서 <b>제외하고 싶은 요소</b>를 벡터 공간에서 멀어지게 설정</td>
        </tr>
        <tr>
            <td><b>guidance_scale</b></td>
            <td>unet</td>
            <td>AI가 프롬프트를 얼마나 <b>강하게(고집스럽게)</b> 따를지 결정</td>
        </tr>
        <tr>
            <td><b>num_inference_steps</b></td>
            <td>scheduler</td>
            <td>노이즈를 제거하는 <b>반복 횟수</b>. 많을수록 정교하지만 시간이 오래 걸림</td>
        </tr>
        <tr>
            <td><b>generator (seed)</b></td>
            <td>Latent Noise</td>
            <td>이미지의 기초가 되는 <b>난수(노이즈) 패턴</b>을 고정 (동일 구도 재현)</td>
        </tr>
        <tr>
            <td><b>width / height</b></td>
            <td>unet, vae</td>
            <td>출력 이미지의 <b>해상도</b>. VRAM 사용량과 구도(가로/세로)에 영향</td>
        </tr>
    </tbody>
</table>

---
## 2) 텍스트 프롬프트로 이미지 생성
- **목적:** 텍스트 프롬프트를 입력받아 그에 맞는 이미지를 생성합니다.
- **관찰 포인트**
    - prompt는 텍스트 설명으로, 해당 설명에 맞는 이미지를 생성
    - `guidance_scale`은 모델이 텍스트 프롬프트를 얼마나 "강력하게" 반영할지를 결정하는 매개변수임

In [ ]:
# 창의성(높을수록 창의적, 낮을수록 구체적)
guidance_scale = 7.5

# 생성할 이미지에 대한 텍스트 프롬프트 정의
prompt = "A futuristic cityscape at sunset with flying cars"

# 이미지 생성
image = pipe(prompt, guidance_scale=guidance_scale).images[0]
image

In [ ]:
prompt = "An astronaut riding a horse in a photorealistic style"
image = pipe(prompt, guidance_scale=guidance_scale).images[0]
image

---
## 3) 창의성 변경
- **목적:** guidance_scale을 변경하면서 결과 이미지를 확인합니다.
- **관찰 포인트**
    - guidance_scale은 모델이 프롬프트(텍스트)를 얼마나 강하게 따를 것인지를 결정하는 중요한 파라미터임.
    - 값이 낮으면 프롬프트에 매우 엄격한 이미지가 나오고, 값이 높으면 자유롭고 창의적인(하지만 엉뚱한) 이미지가 나옴.

In [ ]:
import torch
import matplotlib.pyplot as plt

# 1. 설정
prompt = "A high-quality fantasy landscape, a mystical river flowing through a neon forest"
scales = [1.5, 10.0, 20.0]  # 비교할 세 가지 값 (낮음, 보통, 높음)
images = []

# 2. 반복문을 통해 이미지 생성
print(f"이미지 생성 시작: {prompt}")
for scale in scales:
    print(f"현재 작업 중: guidance_scale = {scale}")
    # 이미지 생성 (추론 속도를 위해 도중에 진행 상황 표시 생략 가능)
    generator = torch.manual_seed(50) # 동일한 조건 비교를 위해 시드 고정
    result = pipe(prompt, guidance_scale=scale, generator=generator).images[0]
    images.append((scale, result))

# 3. 결과 시각화 (Matplotlib 사용)
plt.figure(figsize=(20, 7)) # 전체 창 크기 설정

for i, (scale, img) in enumerate(images):
    plt.subplot(1, 3, i + 1) # 1행 3열 중 i+1번째
    plt.imshow(img)
    plt.title(f"Guidance Scale: {scale}", fontsize=15)
    plt.axis("off") # 축 정보 숨기기

plt.tight_layout()
plt.show()

---
## 3-1) 과도한 창의성
- guidance_scale을 **매우 낮게(1.0)**부터 **매우 높게(30.0)**까지 설정해 봄
- 수치가 너무 높을 때 이미지가 어떻게 깨지는지(Over-baked) 직접 관찰해 봄

In [ ]:
prompt = "A hyper-realistic portrait of a golden retriever wearing sunglasses"
cfg_values = [1.0, 7.5, 30.0] # 1: 자유로움, 7.5: 권장, 30: 과함
images = []

for cfg in cfg_values:
    generator = torch.manual_seed(500)
    img = pipe(prompt, guidance_scale=cfg, generator=generator).images[0]
    images.append((cfg, img))

plt.figure(figsize=(18, 6))
for i, (val, img) in enumerate(images):
    plt.subplot(1, 3, i+1)
    plt.imshow(img); plt.title(f"Guidance Scale: {val}"); plt.axis("off")
plt.show()

---
## 4) 시드 변경
- **목적:** Seed를 변경하면서 결과 이미지를 확인합니다.
- **관찰 포인트**
    - 프롬프트가 같아도 '랜덤 노이즈(Seed)'가 바뀌면 완전히 다른 구도의 이미지가 생성됨
    - 어떤 시드 번호가 만족스러운 결과를 내는지 찾는 과정을 체험함

In [ ]:
import torch
import matplotlib.pyplot as plt

prompt = "A majestic cat wearing a king's crown, oil painting style"
seeds = [123, 456, 789] # 무작위 시드 번호
images = []

for seed in seeds:
    # 동일한 시드 설정을 위해 generator 생성
    generator = torch.Generator("cuda").manual_seed(seed)
    image = pipe(prompt, generator=generator).images[0]
    images.append((seed, image))

# 결과 출력
plt.figure(figsize=(15, 5))
for i, (seed, img) in enumerate(images):
    plt.subplot(1, 3, i+1)
    plt.imshow(img)
    plt.title(f"Seed: {seed}")
    plt.axis("off")
plt.show()

---
## 5) Negative Prompt 반영
- **목적:** 결과 이미지에서 원하지 않는 요소들을 제거해 봅니다.
- **관찰 포인트**
    - "이것만은 그리지 마"라고 지시하는 기법.
    - 선호하는 이미지를 얻는 효과적인 방법 중 하나 임.

In [ ]:
import torch
import matplotlib.pyplot as plt

# 1. 프롬프트 설정 (긍정 프롬프트는 동일하게 유지)
# 빨간 꽃이 가득한 숲속에 한 여성이 서 있는 수채화
# prompt = "A beautiful watercolor painting of a lush green forest with many bright red flowers, a woman standing in the center, high quality"
prompt = "A beautiful watercolor painting of a green forest with many bright red flowers, a woman standing in the center, high quality"

# 2. 네거티브 프롬프트 설정
# 첫 번째는 아무것도 제한하지 않음, 두 번째는 '사람'과 '빨간색'을 강력하게 제한
negative_prompts = [
    "",
    "woman, female, person, red flowers, red color"
]

images = []
generator_seed = 2024 # 차이를 확실히 보기 위해 구도 고정

for neg_prompt in negative_prompts:
    generator = torch.Generator("cuda").manual_seed(generator_seed)

    # 이미지 생성
    image = pipe(
        prompt,
        negative_prompt=neg_prompt,
        generator=generator,
        num_inference_steps=30,
        guidance_scale=7.5
    ).images[0]

    images.append((neg_prompt, image))

# 3. 결과 비교 시각화
fig, ax = plt.subplots(1, 2, figsize=(16, 8))

titles = ["No Negative Prompt (Original)", "With Negative Prompt\n(No Woman, No Red)"]

for i in range(2):
    ax[i].imshow(images[i][1])
    ax[i].set_title(titles[i], fontsize=15, pad=20)
    ax[i].axis("off")

plt.tight_layout()
plt.show()

---
## 6) Inference Steps 변경
- **목적:** 결과 이미지 생성 단계를 조절하면서 결과 이미지 변화를 확인합니다.
- **관찰 포인트**
    - 그림을 얼마나 정성 들여 그릴지(단계)를 조절함
    - 단계가 높을수록 디테일해지지만 시간이 오래 걸림

In [ ]:
prompt = "A futuristic cyberpunk city, ultra detailed"
steps_list = [5, 20, 100] # 단계 설정
images = []

for steps in steps_list:
    generator = torch.manual_seed(1004) # 비교를 위해 시드 고정
    image = pipe(prompt, num_inference_steps=steps, generator=generator).images[0]
    images.append((steps, image))

# 결과 출력
plt.figure(figsize=(15, 5))
for i, (steps, img) in enumerate(images):
    plt.subplot(1, 3, i+1)
    plt.imshow(img)
    plt.title(f"Steps: {steps}")
    plt.axis("off")
plt.show()

---
## 7) Style Variation (화풍 전환)
- **목적:** 결과 이미지의 화풍을 변경시켜 봅니다.
- **관찰 포인트**
    - 동일한 대상을 두고 '스타일' 프롬프트만 바꿔서 화풍 변화를 체험함.

In [ ]:
# 동일한 대상, 다른 스타일
base_prompt = "A futuristic sports car parked in a rainy city street"
styles = [
    "Cyberpunk style, neon lights, 8k resolution",
    "Oil painting, thick brushstrokes, van Gogh style",
    "Blueprint, technical drawing, architectural sketch"
]

images = []
for style in styles:
    full_prompt = f"{base_prompt}, {style}"
    # 일관성을 위해 시드 고정
    generator = torch.manual_seed(1234)
    img = pipe(full_prompt, generator=generator).images[0]
    images.append((style.split(',')[0], img))

# 결과 출력 (3분할)
plt.figure(figsize=(18, 6))
for i, (title, img) in enumerate(images):
    plt.subplot(1, 3, i+1)
    plt.imshow(img); plt.title(title); plt.axis("off")
plt.show()

---
## 8) 해상도(Resolution)와 구도의 관계
- **목적:** 결과 이미지의 해상도와 구도간의 상관 관계를 알아봅니다.
- **관찰 포인트**
    - Stable Diffusion v1-4는 기본 512x512 학습 모델임.
    - 가로/세로 길이를 바꿨을 때 구도가 어떻게 변하는지 확인함 (VRAM 용량이 작다면 조심해야 함)

In [ ]:
prompt = "A full body shot of a medieval knight standing in a vast desert"
# (가로, 세로) 조합
sizes = [(512, 512), (768, 512), (512, 768)]
images = []

for w, h in sizes:
    generator = torch.manual_seed(10)
    # width와 height 파라미터 사용
    img = pipe(prompt, width=w, height=h, generator=generator).images[0]
    images.append((f"{w}x{h}", img))

plt.figure(figsize=(18, 6))
for i, (label, img) in enumerate(images):
    plt.subplot(1, 3, i+1)
    plt.imshow(img); plt.title(label); plt.axis("off")
plt.show()

---
## (Optional) VRAM 체크
- 현재 내 그래픽카드의 메모리가 얼마나 남았는지 확인하는 방법
- CUDA Out of Memory (VRAM 부족) 대응 필요

In [ ]:
import torch

def check_vram():
    if torch.cuda.is_available():
        # 현재 사용 중인 장치의 인덱스
        device = torch.cuda.current_device()
        # 장치 이름 (예: Tesla T4, RTX 3060 등)
        device_name = torch.cuda.get_device_name(device)

        # 전체 VRAM
        total_memory = torch.cuda.get_device_properties(device).total_memory / 1024**3
        # 현재 할당된 VRAM
        allocated_memory = torch.cuda.memory_allocated(device) / 1024**3
        # 캐시된(예약된) VRAM
        reserved_memory = torch.cuda.memory_reserved(device) / 1024**3
        # 실제 여유 공간
        free_memory = total_memory - reserved_memory

        print(f"--- GPU 정보: {device_name} ---")
        print(f"전체 VRAM: {total_memory:.2f} GB")
        print(f"사용 중인 VRAM: {allocated_memory:.2f} GB")
        print(f"예약된 VRAM: {reserved_memory:.2f} GB")
        print(f"여유 VRAM (여유분): {free_memory:.2f} GB")
    else:
        print("GPU를 사용할 수 없는 환경입니다.")

check_vram()

---
## (Optional) VRAM 부족시 대처
- VRAM 에러가 났을 때, 런타임을 재시작하지 않고 메모리를 비워주는 코드

In [ ]:
import gc

def clean_memory():
    # 1. 파이썬 객체 참조 해제
    gc.collect()
    # 2. PyTorch 캐시 비우기
    torch.cuda.empty_cache()
    print("VRAM 캐시가 정리되었습니다.")

# 사용 예시
# del pipe  # 만약 모델 자체를 지우고 싶다면 객체 삭제 후 실행
clean_memory()